# Capstone Project: Comment Subtopics Analysis for Airbnb Hosts
---

How can a host on Airbnb understand that are their strengths and weaknesses? How can hosts point out the demand trend of their customers from a large scale of comments? This project focuses on using machine learning tools to help hosts understand the underlying trends of the comments on their property.  

---


# Part 5: Use Comments to Predict Star Ratings For Each Host

- After performing exploratory data analysis on the documents, we can see that here is some kind of realtionship between comments and review score.Now, with this finding, in this documentation, we are going to use comments in each listings to predict the rating for this listing. 
---

### Steps 
1. creating individual corpus for each listing 
2. build a baseline model for prediction 
3. improve on model

In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics

from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn import svm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

np.random.seed(42)

In [2]:
reviews = pd.read_csv('../data/reviews_sentiment_score.csv', index_col = 0)

In [3]:
reviews.head()

,key_0,listing_id,id,date,reviewer_id,reviewer_name,comments,language,overall_rating,compound,neg,neu,pos
0,0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st...",en,97.0,0.9590,0.000,0.788,0.212
1,1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...,en,97.0,0.9819,0.000,0.697,0.303
2,2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...,en,97.0,0.7600,0.134,0.710,0.156
3,3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...,en,97.0,0.9840,0.035,0.646,0.319
4,4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...,en,97.0,0.9617,0.000,0.613,0.387


In [70]:
listing = pd.read_csv('../data/listings/2019-03-06_data_listings.csv')

In [4]:
reviews.dtypes

key_0               int64
listing_id          int64
id                  int64
date               object
reviewer_id         int64
reviewer_name      object
comments           object
language           object
overall_rating    float64
compound          float64
neg               float64
neu               float64
pos               float64
dtype: object

## Creating individual Corpus For Each Listings 

---

In [81]:
review_dict = {}
for i in reviews['listing_id'].unique(): 
    comment_list = list(reviews[reviews['listing_id'] == i]['comments'])
    one_list = "".join(comment_list)
    review_dict[i] = one_list

In [82]:
len(review_dict)

4199

In [83]:
len(reviews['listing_id'].unique())

4199

In [84]:
review_prediction_df = pd.DataFrame(data = [review_dict.keys()])

In [85]:
review_prediction_df = review_prediction_df.T

In [86]:
review_prediction_df.columns = ['listing_id']

In [87]:
review_prediction_df['comment'] = review_prediction_df['listing_id'].map(review_dict)

In [88]:
review_prediction_df['overall_score'] = listing['review_scores_rating']

In [89]:
#complete Review Prediction Data Frame 
review_prediction_df.head()

,listing_id,comment,overall_score
0,958,"Our experience was, without a doubt, a five st...",97.0
1,5858,We had a fabulous time staying with Philip and...,98.0
2,7918,My stay was fantastic! The neighborhood is gr...,85.0
3,8142,"Excellent! The space is clean and quiet, and t...",93.0
4,8339,My stay was wonderful in many ways; the apartm...,97.0


## Cleaning Comments for CountVectorization 

---

In [94]:
def clean_text(text): 
    '''
    This clean_text function will focus on: 
    1. cleaning the content including removing prentecisis, \r\n, : and so on. 
    2. lower case all words.
    3. remove stop words from articles. 
    '''
    text = re.sub(r'\r\n', r' ', text)
    text = re.sub(r'[\\\.\:\*/\,\!]', r' ', text)
    text = re.sub(r'[\(\)]', r' ', text)
    text = re.sub(r'[\"\“\”\—\[\]]', r' ', text)
    text = re.sub(r"'s", r' ', text)
    text = re.sub(r'[\\"]', r' ', text)
    text = text.lower()
    
    return text

In [95]:
clean_comment = []
for comment in review_prediction_df['comment']: 
    new_comment = clean_text(comment)
    clean_comment.append(new_comment)

In [96]:
review_prediction_df['comment'] = clean_comment

In [97]:
review_prediction_df.head()

,listing_id,comment,overall_score
0,958,our experience was without a doubt a five st...,97.0
1,5858,we had a fabulous time staying with philip and...,98.0
2,7918,my stay was fantastic the neighborhood is gr...,85.0
3,8142,excellent the space is clean and quiet and t...,93.0
4,8339,my stay was wonderful in many ways; the apartm...,97.0


## Train Test Split 
___

In [136]:
X = review_prediction_df['comment']
y = review_prediction_df['overall_score'].fillna(0)

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [138]:
#building baseine model 
cv = CountVectorizer(ngram_range= (2,2), 
                     stop_words= 'english', 
                     min_df = 2)
X_train_cv = cv.fit_transform(X_train)

In [139]:
X_test_cv = cv.transform(X_test)

In [140]:
X_train_cv.shape, X_test_cv.shape

((3149, 356357), (1050, 356357))

## Dimention Reduction 
---

In [156]:
svd = TruncatedSVD(n_components= 500, random_state= 42)
svd.fit(X_train_cv)

TruncatedSVD(algorithm='randomized', n_components=500, n_iter=5,
       random_state=42, tol=0.0)

In [157]:
X_train_svd = svd.transform(X_train_cv)
X_test_svd = svd.transform(X_test_cv)

In [158]:
X_train_svd.shape, X_test_svd.shape

((3149, 500), (1050, 500))

## Building Baseline Model 
---

In [167]:
#without dimention Reduction
lr = LinearRegression()
lr.fit(X_train_cv, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [168]:
print(lr.score(X_train_cv, y_train))
lr.score(X_test_cv, y_test)

0.9911303292592297


-6.290357151105631

In [159]:
#with Dimention Reduction
lr = LinearRegression()
lr.fit(X_train_svd, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [160]:
print(lr.score(X_train_svd, y_train))
lr.score(X_test_svd, y_test)

0.09535843428100454


-0.020430016123151873

In [ ]:
#Random Forest 
#without dimention Reduction
rf = RandomForestRegressor(n_estimators= 50, max_depth= 100)

rf.fit(X_train_cv, y_train)

print(rf.score(X_train_cv, y_train))
print(rf.score(X_test_cv, y_test))